# Finding the right spot to open a gym
**In Treviso province, Italy**

In [10]:
import requests
import pandas as pd
from pandas import json_normalize
import folium
import numpy as np

print('Libraries imported!')

Libraries imported!


First, we are going to import municipalities and province data

In [12]:
# data about all Italy provinces and municipalities
df = pd.read_excel('Dati comunali e provinciali.xlsx',
                   sheet_name = 'Dati comunali',
                   usecols = ['Codice Provincia','Denominazione Comune', 'Popolazione residente al Censimento 2011'])

# filtering to keep only Treviso province
tv = df[df['Codice Provincia'] == 26]
tv = tv.drop('Codice Provincia', 1)
tv.columns = ['Municipality', 'Population']

tv.head()

,Municipality,Population
3445,Altivole,6720.0
3446,Arcade,4366.0
3447,Asolo,8952.0
3448,Borso del Grappa,5913.0
3449,Breda di Piave,7750.0


Then, we will build the connection to Foursquare APIs to get venues data

In [16]:
# Treviso province coordinates
tv_lat = 45.808113910038976
tv_lon = 12.222609501822564

# connection data
CLIENT_ID = 'xxxxxx'
CLIENT_SECRET = 'xxxxxx'
VERSION = '20180605' 
LIMIT = 100

# we are going to look only for the "father" category "Gym", which has some subcategories, so we specify category ID
CATEGORY_ID = '4bf58dd8d48988d175941735'

# large radius to get all the province and some venues in the immediate outsides of it
radius = 45000

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, tv_lat, tv_lon, VERSION, radius, LIMIT, CATEGORY_ID)

results = requests.get(url).json()

Transforming Foursquare json into a dataframe

In [17]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']

# flatten json
nearby_venues = json_normalize(venues) 

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,JOHN REED Fitness,Gym / Fitness Center,45.491120,12.237970
1,Virgin Active,Gym / Fitness Center,45.510662,12.235397
2,McFIT Treviso Ducale,Gym,45.653574,12.228526
3,Rosa Blu Village,Gym / Fitness Center,45.648691,12.409053
4,Blufit Healt & Fitness,Gym,45.785069,12.050451


We want to use Folium to visualize where the existing gyms are

In [18]:
map1 = folium.Map(location = [tv_lat, tv_lon], 
                  zoom_start = 9)

# add gym markers to map
for lat, lng, name in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    label = name
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map1) 

map1

We also want some more info on the province and its municipalities, so we visualize a choropleth map of the municipalities, colored according to their population, to better understand where there are more inhabitants

In [19]:
# we need geospatial data to make this map
comuni_geo = r'treviso.geojson'

# set thresholds to better visualize municipalities population differences
threshold_scale = [790, 5000, 10000, 20000, 40000, 81015]

map2 = folium.Map(location = [tv_lat, tv_lon], 
                  zoom_start = 10)

map2.choropleth(geo_data = comuni_geo, 
              data = tv,
              columns = ['Municipality', 'Population'],
              key_on = 'feature.properties.name',
              threshold_scale = threshold_scale,
              fill_color = 'YlOrRd', 
              fill_opacity = 0.7, 
              line_opacity = 0.2,
              legend_name = 'Municipality population')
               
folium.LayerControl().add_to(map2)

map2

Now we can mix both maps and see at the same time where most people live and where we already have gyms in the area

In [20]:
map3 = folium.Map(location = [tv_lat, tv_lon], 
                  zoom_start = 10)

# choropleth map with municipalities
map3.choropleth(geo_data = comuni_geo, 
              data = tv,
              columns = ['Municipality', 'Population'],
              key_on = 'feature.properties.name',
              threshold_scale = threshold_scale,
              fill_color = 'YlOrRd', 
              fill_opacity = 0.7, 
              line_opacity = 0.2,
              legend_name = 'Municipality population')
               
folium.LayerControl().add_to(map3)

# adding gym markers
for lat, lng, name in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    label = name
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map3) 

map3

This tells us quite a bit, and gives us two initial ideas of potential good areas to open a new gym:
- one is around Valdobbiadene municipality (upper-left part of the province), since there is a quite large area with no gyms there, and Valdobbiadene itself is a slighly populated municipality)
- the other one is south of Conegliano, that zone has not many gyms and Conegliano is one of the most populated areas in the province

We are going to add the two proposed areas on a new map

In [22]:
map4 = folium.Map(location = [tv_lat, tv_lon], 
                  zoom_start = 10)

# choropleth map
map4.choropleth(geo_data = comuni_geo, 
              data = tv,
              columns = ['Municipality', 'Population'],
              key_on = 'feature.properties.name',
              threshold_scale = threshold_scale,
              fill_color = 'YlOrRd', 
              fill_opacity = 0.7, 
              line_opacity = 0.2,
              legend_name = 'Municipality population')
               
folium.LayerControl().add_to(map4)

# adding gym markers
for lat, lng, name in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    label = name
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map4) 

# adding proposed area circles
folium.CircleMarker(
        [45.865231647682165, 12.280513713773814],
        radius = 30,
        color = 'green',
        fill = False,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map4) 

folium.CircleMarker(
        [45.90988072432466, 12.039426473166502],
        radius = 30,
        color = 'green',
        fill = False,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map4) 

map4